In this notebook we will learn how to create and train an embedding layer for the words appearing in a text data. We will then train a simple DNN based model to do sentiment analysis on this data. 

## Exercise

This is exercise 13.10 in [this](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) book.

### Problem Statement

In this exercise you will download a dataset, split it, create a tf.data.Dataset to load it and preprocess it efficiently, then build and train a binary classification model containing an Embedding layer:

  - a. Download the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/), which contains 50,000 movies reviews from the Internet Movie Database. The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise.
  
    
  - b. Split the test set into a validation set (15,000) and a test set (10,000).
  
  
  - c. Use tf.data to create an efficient dataset for each set.
  
  
  - d. Create a binary classification model, using a TextVectorization layer to preprocess each review. If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method.
  
  
  - e. Add an Embedding layer and compute the mean embedding for each review, multiplied by the square root of the number of words (see Chapter 16). This rescaled mean embedding can then be passed to the rest of your model.
  
  
  - f. Train the model and see what accuracy you get. Try to optimize your pipelines to make training as fast as possible.


  - g. Use TFDS to load the same dataset more easily: tfds.load("imdb_reviews").

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
print('tensorflow version: {}'.format(tf.__version__))
print('keras version: {}'.format(keras.__version__))

tensorflow version: 2.1.0
keras version: 2.2.4-tf


In [2]:
import os
print('cwd: {}'.format(os.getcwd()))

cwd: /home/prarit/MachineLearningProjects/Word-Embeddings


### Downloading the Large Movie Review Dataset

In [3]:
# good tutorial on using wget: https://www.tecmint.com/download-and-extract-tar-files-with-one-command/
# turn off verbose output of wget using the flag -nv : https://shapeshed.com/unix-wget/#how-to-turn-off-verbose-output 
!wget -c -nv http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -o - 

In [4]:
# uncompress the downloaded files
!tar xzf aclImdb_v1.tar.gz

Note that tensorflow also provides this dataset: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb

### Briefly Explore the Dataset

In [5]:
# list the files in the current working directory
os.listdir()

['README.md',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 'aclImdb',
 'Word-Embeddings.ipynb',
 'aclImdb_v1.tar.gz']

We see that aclImdb_v1.tar.gz was extracted to a folder called aclImdb. Let's see the contents of this file.

In [6]:
path = os.path.join(os.getcwd() , 'aclImdb')
contents = os.listdir(path)
print('The contents of aclImdb are: \n{}'.format(contents))

The contents of aclImdb are: 
['imdb.vocab', 'train', 'README', 'imdbEr.txt', 'test']


There is a README file in aclImdb, let us read it.

In [7]:
# read README
filepath = os.path.join(path, 'README')
with open(filepath, 'r') as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

From README, we find that the train/test folder contains a 'pos' folder for positive reviews and a 'neg' for negative reviews along with .txt files containing urls of positive and negative reviews respectively. There are also some other files for bag-of-words features etc. 

Each train/test folder contains a total of 25000 reviews of which 12500 are positive reviews and 12500 are negative reviews.

Let's verify the above about the 'train' folder

In [8]:
train_path = os.path.join(path,'train')
print('contents of the train folder: \n{}'.format(os.listdir(train_path)))

contents of the train folder: 
['unsup', 'urls_pos.txt', 'pos', 'labeledBow.feat', 'unsupBow.feat', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


In order to create a dataset, we need the path to all the reviews. We can create the corresponding list of paths using glob

In [9]:
import glob

In [10]:
# paths to the positive reviews in the training set
train_pos_path = os.path.join(train_path, 'pos', '*.txt')
train_pos_reviews = glob.glob(train_pos_path)
print('No. of train-set files with positive reviews: {}'.format(len(train_pos_reviews)))

No. of train-set files with positive reviews: 12500


In [11]:
# paths to the negative reviews in the training set
train_neg_path = os.path.join(train_path, 'neg', '*.txt')
train_neg_reviews = glob.glob(train_neg_path)
print('No. of train-set files with negative reviews: {}'.format(len(train_neg_reviews)))

No. of train-set files with negative reviews: 12500


let us give a brief look at a positive review. 

In [12]:
file = train_pos_reviews[0]
with open(file, 'r') as f:
    print(f.read())

I watch them all.<br /><br />It's not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it's completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don't watch. 'How she move.' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It's just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I'm a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn't all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more clearly set there. I 

Later, we will like to load an preprocess all the data using tensorflow's data API, therefore let us quickly see how to read the same file as above but this time by using tensorflow's [TextLineDataset](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) method.

In [13]:
pos_fl0 = tf.data.TextLineDataset(file)
for item in pos_fl0:
    print(item)

tf.Tensor(b'I watch them all.<br /><br />It\'s not better than the amazing ones (_Strictly Ballroom_, _Shall we dance?_ (Japanese version), but it\'s completely respectable and pleasingly different in parts.<br /><br />I am an English teacher and I find some of the ignorance about language in some of these reviews rather upsetting. For example: the "name should scream don\'t watch. \'How she move.\' Since when can movie titles ignore grammar?" <br /><br />There is nothing inherently incorrect about Caribbean English grammar. It\'s just not Canadian standard English grammar. Comments about the dialogue seem off to me. I put on the subtitles because I\'m a Canadian standard English speaker, so I just AUTOMATICALLY assumed that I would have trouble understanding all of it. It wasn\'t all that difficult and it gave a distinctly different flavour as the other step movies I have seen were so American.<br /><br />I loved that this movie was set in Toronto and, in fact, wish it was even more c

Perfect! As expected, we see that the pos_fl0 contains a single item and it's value output matches the output of the previous code cell. 

### Preprocessing the reviews

Having, learnt how to use tf.data.TextLineData() method, we can now starting preprocessing the data. In order to do this, we notice that the review contains punctuation marks and html line break tags etc. We will have to write a preprocessing function to get rid of these. Additionally, we will also change all alphabets to lower case.

#### Removing line-brk tags

This can be very simply done by using the .replace() method of python strings. We can therefore use it to replace all occurrances of the line-break tag with a space. In tensorflow, the equivalent method is [tf.strings.regex_replace()](https://www.tensorflow.org/api_docs/python/tf/strings/regex_replace).

#### tf.strings.regex_replace
Note that 'regex' in regex_replace() stands for ["regular expression"](https://docs.python.org/3/howto/regex.html). For e.g. the following will work:

In [14]:
tf.strings.regex_replace('hello','e','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

But the following will throw an error: tf.strings.regex_replace('h(llo', '(','E')

This is because "(" is a metacharacter. To match and replace meta-characters, we must prepend a backslash before them. This can be done as follows: '\\\\' + char. 

The previous code cell can now be made to work

In [15]:
# The error thrown by this code cell is intentional
tf.strings.regex_replace('h(llo', '\\'+'(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [16]:
tf.strings.regex_replace('h(llo','\(','E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

##### Imp: 

Note that backslash itself is also a  meta-character. To search and replace backslash, we do the following:

In [17]:
tf.strings.regex_replace('h\llo', '\\'+'\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

or equivalently:

In [18]:
# we use '\\\\' and NOT '\\' to search and replace a backslash
tf.strings.regex_replace('h\llo', '\\\\', 'E')

<tf.Tensor: shape=(), dtype=string, numpy=b'hEllo'>

We now try three different ways of removing punctuations from a tensorflow string and compare their timings:

1) punc_filter_and_to_lower1: use tf.strings.unicode_decode() to convert all the characters in the string into an array of their ascii codes. We then loop through this array, skipping over the places where the entry matches the ascii code of a punctuation. Finally we call tf.strings.unicode_encode() on this array to convert the ascii codes back to characters, thereby obtaining a string with all the punctuations stripped. 


2) punc_filter_and_to_lower2: use tf.regex_replace() to search and replace each punctuation by empty space. Pay special attention to prepend backslash in order to be able to use meta-characters in regex_replace.


3) punc_filter_and_to_lower3: simply extract the python string using its .numpy() method. Then simple iterate through the characters of the string, skipping over the punctutations. Join the resulting list of character using .join() method. 

4) punc_filter_and_to_lower4: Use [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) by setting it's "alphanum_only" arg to True. This way it will only parse alpha-numeric characters in the text and to split the text at occurrances of non-alphanumeric characters. Since whitespace is a non-alpha-numeric character, the output will largely consist of a list of words in the text.  The caeat with this approach is that words with apostrophe in them such as " don't " will be split into two words: "don" and "t". Also, [tfds.features.text.Tokenizer()](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) does NOT consider underscores as non-alpha-numeric, so underscores do not get removed form the text. On the other hand since, the tokenizer already generates a list of words, we use this list to generate a vocobulary of words in the dataset at this step it-self, making the preprocessing faster.

There timing on the first review in the training set was as follows:

1) punc_filter_and_to_lower1: Wall time: ~ 3 s

2) punc_filter_and_to_lower2: Wall time: ~ 8 ms

3) punc_filter_and_to_lower3: Wall time: ~ 6 ms

4) punc_filter_and_to_lower4: Wall time: ~ 6 ms (when not udating a vocabulary of words) 

5) 4) punc_filter_and_to_lower4: Wall time: ~ 7 ms (when also udating a vocabulary of words) 

Clearly, the last function is the fastest with the first one being extremely slow (takes several secs). 

In [19]:
import string

In [20]:
# list of punctuations
punc_ls = string.punctuation
print('punctuations before utf encoding: {}'.format(punc_ls))
punc_ls2 = tf.strings.unicode_decode(punc_ls, input_encoding = 'UTF-8')
print('punctuations after utf encoding: \n{}'.format(punc_ls2))

punctuations before utf encoding: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punctuations after utf encoding: 
[ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  58  59  60
  61  62  63  64  91  92  93  94  95  96 123 124 125 126]


In [21]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.unicode_encode(), tf.strings.unicode_decode()
def punc_filter_and_to_lower1(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = tf.strings.unicode_decode(st2, input_encoding = 'utf-8')
    st2 = tf.strings.unicode_encode([char for char in st2 if char not in punc_ls2], output_encoding = 'UTF-8')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [22]:
import time

In [23]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower1(item)

CPU times: user 3.47 s, sys: 47.5 ms, total: 3.51 s
Wall time: 3.54 s


In [24]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [25]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tf.strings.regex_replace()
def punc_filter_and_to_lower2(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ')
    
    # now we replace all the punctions in the string
    for punc in punc_ls:
        # V.imp: to replace meta-characters we prepend a backslash to them 
        # infact we can also prepend a backslach before all the characters
        # to prepend a backslash before a character we do: '\\' + char
        st2 = tf.strings.regex_replace(st2, '\\'+ punc, ' ')
    
    st2 = tf.strings.lower(st2)
    
    return st2

In [26]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower2(item)

CPU times: user 5.02 ms, sys: 4.94 ms, total: 9.97 ms
Wall time: 9.26 ms


In [27]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all  it s not better than the amazing ones   strictly ballroom    shall we dance    japanese version   but it s completely respectable and pleasingly different in parts  i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting  for example  the  name should scream don t watch   how she move   since when can movie titles ignore grammar    there is nothing inherently incorrect about caribbean english grammar  it s just not canadian standard english grammar  comments about the dialogue seem off to me  i put on the subtitles because i m a canadian standard english speaker  so i just automatically assumed that i would have trouble understanding all of it  it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american  i loved that this movie was set in toronto and  in fact  wish it was even more clearly set there  i 

In [28]:
# function to get rid of punctuations, html line breaks and change to lower case
# using .join() method in python string class
def punc_filter_and_to_lower3(st):
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.regex_replace(st, line_brk_tag, ' ') # regex stands for regular expressions
                                                          # i.e. they are not metacharacters
                                                          # https://docs.python.org/3/howto/regex.html#matching-characters
    
    # now we replace all the punctions in the string
    st2 = st2.numpy().decode('utf-8')
    st2 = ''.join([char for char in st2 if char not in punc_ls])
    
    st2 = st2.lower()
    
    st2 = tf.convert_to_tensor(st2)
    
    return st2

In [29]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower3(item)

CPU times: user 5.14 ms, sys: 203 µs, total: 5.34 ms
Wall time: 5.01 ms


In [30]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all its not better than the amazing ones strictly ballroom shall we dance japanese version but its completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream dont watch how she move since when can movie titles ignore grammar  there is nothing inherently incorrect about caribbean english grammar its just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because im a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasnt all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was so atypic

In [31]:
# function to get rid of punctuations, html line breaks and change to lower case
# using tfds.features.text.tokenizer()
# we will also simultaneously generate a vocabulary in this step
import tensorflow_datasets as tfds
def punc_filter_and_to_lower4(st, vocab = None):
    ''' vocab: vocabulary to update with the words in st'''
    
    # before removing punctuations, we should remove the html line-break tag
    # this is because the line break tag contains <,/ and > characters which 
    # will be removed if we remove punctuations first. This will then make it harder 
    # to identify the line-break-tag
    line_brk_tag = "<br /><br />" 
    st2 = tf.strings.lower(tf.strings.regex_replace(st, line_brk_tag, ' '))
    
    # now we replace all the punctions in the string
    tokenizer = tfds.features.text.Tokenizer()
    words = tokenizer.tokenize(st2.numpy()) # note that the inpout has to be a python string NOT as tensor 
                                            # The output is a list NOT a tensor
                                            # https://stackoverflow.com/questions/56665868/tensor-numpy-not-working-in-tensorflow-data-dataset-throws-the-error-attribu
    st2 = tf.strings.join(words, ' ')
    
    if type(vocab) == set:
        vocab.update(words)
        return st2, vocab
    
    
    return st2

In [32]:
%%time
for item in pos_fl0:
    lst = punc_filter_and_to_lower4(item)

CPU times: user 7.72 ms, sys: 0 ns, total: 7.72 ms
Wall time: 6.96 ms


In [33]:
lst

<tf.Tensor: shape=(), dtype=string, numpy=b'i watch them all it s not better than the amazing ones _strictly ballroom_ _shall we dance _ japanese version but it s completely respectable and pleasingly different in parts i am an english teacher and i find some of the ignorance about language in some of these reviews rather upsetting for example the name should scream don t watch how she move since when can movie titles ignore grammar there is nothing inherently incorrect about caribbean english grammar it s just not canadian standard english grammar comments about the dialogue seem off to me i put on the subtitles because i m a canadian standard english speaker so i just automatically assumed that i would have trouble understanding all of it it wasn t all that difficult and it gave a distinctly different flavour as the other step movies i have seen were so american i loved that this movie was set in toronto and in fact wish it was even more clearly set there i loved that the heroine was

In [34]:
%%time
# test the timing of punc_filter_and_to_lower4 when simultaneously updating a dictionary
vocab2 = set([])
for item in pos_fl0:
    lst, vocab2 = punc_filter_and_to_lower4(item, vocab2)

CPU times: user 8.04 ms, sys: 0 ns, total: 8.04 ms
Wall time: 7.54 ms


### Build a Vocabulary based on the training data

This can be easily done using python's [set()](https://docs.python.org/3.8/library/stdtypes.html#set-types-set-frozenset) container.: basically, split each text in the training instance into words and update the set with this list. This will add any new words in the text to the set. In the end, the set will contain all the unique words in the training dataset.

Note that we have also implemented the above idea in our function punc_filter_and_to_lower4() 

In [35]:
# function to create a vocabulary using the set() container
vocab1 = set([]) # python set for containing unique words in the training dataset
def vocab_builder1(strng):
    # split the string into its words
    words = tf.strings.split(strng)
    # update vocab
    vocab1.update(words.numpy())
    return

In [36]:
%%time
for item in pos_fl0:
    item = punc_filter_and_to_lower3(item)
    vocab_builder1(item)

CPU times: user 308 ms, sys: 117 ms, total: 425 ms
Wall time: 762 ms


Note that the vocab_builder1() function as defined above takes about 300 ms. On the otherhand, in punc_filter_and_to_lower4() function,  we were able to get almost the same result (upto the caveats mentioned in the previous section) by using tfds.features.text.Tokenizer(), in about 8 ms. Clearly, there is a difference of few order of magnitude between the time taken by the two routines with the difference between their output being quite tolerable.  

#### building the vocabulary

In [37]:
vocab = set([])
train_filepaths = tf.data.Dataset.list_files([train_pos_path, train_neg_path])
train_dataset = tf.data.TextLineDataset(train_filepaths)

ctr = 0
for item in train_dataset:
    _ , vocab = punc_filter_and_to_lower4(item, vocab)
    
    ctr+=1
    if ctr%255 ==0:
        print("=", end = '')

In [38]:
# pick 5 random words from vocab to inspect it
import random

random.sample(vocab, 5)

['shouldn', 'candelabras', 'megatron', 'cineaste', 'hometown']

In [39]:
print('no. of words in vocabulary: {}'.format(len(vocab)))

no. of words in vocabulary: 74893


In [40]:
word_indices = { item:index for index, item in enumerate(vocab)}
len(word_indices)

74893

### Create a lookup table based on the vocabulary

In [71]:
indices = tf.range(len(vocab), dtype = tf.int64)
lookup_initializer = tf.lookup.KeyValueTensorInitializer( list(vocab), indices) 
num_oov_buckets = 50 
look_table = tf.lookup.StaticVocabularyTable( initializer = lookup_initializer, 
                                               num_oov_buckets = num_oov_buckets)

### Labeling the data

Note that the positive and negative reviews in each dataset have been stored in seperate folders and as such they have no explicit labels. We will therefore have to create our own labels as was done in [this](https://www.tensorflow.org/tutorials/load_data/text) official tensorflow tutorial. 

In [41]:
# function to return apply a label
def labeler(review, label):
    return review, tf.cast(label, tf.int64)

In [42]:
train_pos_paths = tf.data.Dataset.list_files(train_pos_path)
train_pos_reviews = tf.data.TextLineDataset(train_pos_paths).map(lambda x : labeler(x, 1))

In [43]:
for item in train_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"I don't know if this is a sitcom or not, but I agree that this is one of the greatest television shows ever. It's great that this show still airs. And I love Michelle. It's cute on the episodes when she was a baby and she talked, and she sometimes said something funny. Aw.<br /><br />This show can relate to children and teens and.. well, families as they struggle through rough times and try to work it out as a family. I don't know who would ever turn down an opportunity to watch this show with someone. <br /><br />I love the episode when I think her name is DD.. the older girl accidentally stole a sweatshirt, and she learned a lesson about stealing. That was a great episode. An example that this TV show shows the family working things out as a family.<br /><br />I recommend this show for everyone.">, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b"Stuck in a hotel in Kuwait, I happily switched to t

### Creating training batches

In [44]:
train_neg_paths = tf.data.Dataset.list_files(train_neg_path)
train_neg_reviews = tf.data.TextLineDataset(train_neg_paths).map(lambda x: labeler(x, 0))

In [45]:
for item in train_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"All internet buzz aside, this movie was god awful. I expected the movie to be more of a farce than anything. Instead the film makers tried to make a serious thriller/horror movie, and they completely missed. There were only a few good parts, and a couple good lines by Samuel Jackson. Other than that, it was a bunch of gore and some poorly animated snakes. All of the internet joking was miles better than the actual movie. Now that the movie has actually come out, hopefully this joke will die. Don't waste your time or money on this piece of over hyped trash. If you're looking for something that's funny and entertaining, then just go to Snakes on a Blog.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
 
(<tf.Tensor: shape=(), dtype=string, numpy=b'What a stinkeroo this turned out to be!!! At one time, much earlier in her career, Linda Darnell was one of my favorites - no great shakes as an Actress, but very beautiful and pleasant (particularly 

In [46]:
shuffle_buffer_size = 20000
train_dataset = train_pos_reviews.concatenate(train_neg_reviews).shuffle(buffer_size = shuffle_buffer_size)

In [47]:
count = 0
for item in train_dataset:
    count+=1
print(count)    

25000


We now wish to apply punc_filter_and_to_lower4() to the reviews in our training dataset. For this we will use the [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) method of tf.data.Datasets. At this point it is important to note that (as mentioned in the documentation for [map](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map)):

    Note that irrespective of the context in which map_func is defined (eager vs. graph), tf.data traces the function and executes it as a graph. To use Python code inside of the function you have two options:

     1) Rely on AutoGraph to convert Python code into an equivalent graph computation. The downside of this approach is that AutoGraph can convert some but not all Python code.

     2) Use tf.py_function, which allows you to write arbitrary Python code but will generally result in worse performance than 1)

This point is of concern to us because the [tokenizer](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer) we used in punc_filter_and_to_lower4() only accepts string input and NOT tensors. This string was extracted from the tensor using its .numpy() method which is only operable in eager mode. Thus when we pass punc_filter_and_to_lower4() to map(), it throws an error, complaining:

     'Tensor' object has no attribute 'numpy'


Thus we have to wrap punc_filter_and_to_lower4() with tf.py_function before passing it to map


In [48]:
# function to wrap punc_filter_and_to_lower4 with tf.py_function
# we will also use tf.ensure_shape(), or else tensorflow is 
# unable to statically determine it, leading to error being thrown during training
def string_transform(X):
    x = tf.py_function(punc_filter_and_to_lower4, [X], Tout = tf.string)
    x = tf.ensure_shape(x, ())
    
    return x

batch_size = 50
prefetch = 2
train_batch = train_dataset.map(lambda X, y: 
                                    (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

Note that we could have alternatively, preprocessed the data using punc_filter_and_to_lower2() which is based on tensorflow functions and then used it. The price we would have to pay is to rebuild the vocabulary accordingly. 

In [49]:
ite = next(iter(train_batch))
print('shape of train-batch: {}'.format(ite[0].shape))
print(ite[0][0])

shape of train-batch: (50,)
tf.Tensor(b'if i could say it was better than gymkata i at least felt my money was not totally wasted then i saw steven segal s on deadly ground this movie should see a resurrection though on mst 3k if santa claus conquers the martians could make tom servo s head explode one wonders what mayhem this movie could cause there is a very good reason why kurt thomas never had a movie career the writers of this dreck should be forced to wear placards every day of their lives that say bitch slap me i was a writer on gymkata', shape=(), dtype=string)


In [50]:
ite[1]

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1])>

### Create Validation dataset 

As instructed in the excercise, we will create a validation dataset by splitting the test set into half

In [51]:
test_path = os.path.join(path, 'test')
print(test_path)
test_pos_path = os.path.join(test_path, 'pos', '*.txt')
print('path to positive test reviews: \n{}'.format(test_pos_path))
test_neg_path = os.path.join(test_path, 'neg', '*.txt')
print('path to negative test reviews: \n{}'.format(test_neg_path))

/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test
path to positive test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/pos/*.txt
path to negative test reviews: 
/home/prarit/MachineLearningProjects/Word-Embeddings/aclImdb/test/neg/*.txt


In [52]:
test_pos_files = glob.glob(test_pos_path)
len(test_pos_files)

12500

In [54]:
valid_size = 12500
valid_pos_files = random.sample(test_pos_files, int(valid_size/2))
len(valid_pos_files)

6250

In [55]:
# We can remove the valid_pos_files from the set of test_pos_files through the
# simple trick explained in the following post:
# https://stackoverflow.com/questions/6486450/python-compute-list-difference/6486467
test_pos_files = list(set(test_pos_files) - set(valid_pos_files))
len(test_pos_files)

6250

In [56]:
test_neg_files = glob.glob(test_neg_path)
print('initial test_neg_files len:{}'.format(len(test_neg_files)))
valid_neg_files = random.sample(test_neg_files, int(valid_size/2))
print('valid_neg_files len: {}'.format(len(valid_neg_files)))
test_neg_files = list(set(test_neg_files) - set(valid_neg_files))
print('test_neg_files len: {}'.format(len(test_neg_files)))

initial test_neg_files len:12500
valid_neg_files len: 6250
test_neg_files len: 6250


In [57]:
valid_pos_reviews = tf.data.TextLineDataset(valid_pos_files).map(lambda x: labeler(x, 1))
valid_neg_reviews = tf.data.TextLineDataset(valid_neg_files).map(lambda x: labeler(x, 0))

In [58]:
for item in valid_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'The large bell in a bar intermittently rings for last orders and the inevitable rush to queue forms at the counter \xc2\x96 do we want what we need only when it\'s too late? Or is the irony of the opening scene\'s wailing Cassandra a more resonant reflection of our perceptions on individual existence? There\'s an endless fascination about where writer-director Roy Andersson wants to take us in his fourth feature, "You, The Living". With fifty or so semi-related vignettes strung together by a penchant for tragicomic hyper-reality, its wistful interpretations and symbolic instances of life that bind us all in this great big cosmic Sisyphean struggle. The sheer simplicity of these vignettes act to dramatise the tenuity and immense preciousness of being apart of the symbiotic relationships we have with one another. Andersson might whittle down the complexity of the human condition through harsh and fast cynicism more than he should, but he also 

In [59]:
for item in valid_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b"I recall years back, Michael Douglas wanted his wife, Catherine Zeta-Jones, to be in a romantic film because he felt his wife had all the goods. No doubt she does, but NOT in this film. A colossal waste of time, no story, no character development, no chemistry, nada. This was not the vehicle that we all hoped this film would be, boring and a HUGE disappointment. Didn't even watch the whole film, torture. Catherine Zeta-Jones was obviously trained in how to work a kitchen, move around, present a dish but this wasn't the food network, nothing learned here and once her counterpart appeared, supposedly a romantic interest brewing, where was the chemistry. The poor slob on the second floor of her building trying all the ploys to connect and no character development there. The loss of her sister was poorly played out as who knew there was a closeness. The sister's daughter just was plopped here and there with something that was supposed to draw yo

In [60]:
test_pos_reviews = tf.data.TextLineDataset(test_pos_files).map(lambda x: labeler(x, 1))
test_neg_reviews = tf.data.TextLineDataset(test_neg_files).map(lambda x: labeler(x, 0))

In [61]:
for item in test_pos_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'This was a movie that, at the end, I thought "Now that was an enjoyable 2 hours!" I hate spending around $20 (not including baby-sitting $$) for my husband and myself to have that "It was OK" feeling.<br /><br />I think I like Will Smith better as a comedic actor than an action hero. He was well cast in this. His character was very likable, as was Kevin James\'. <br /><br />There were several laugh out loud scenes. It\'s also a romantic movie, so guys, if you want to impress the lady in your life, take her to this. Women will like it as much as any chick-flick, but I wouldn\'t categorize this as one. There is plenty of guy humor in it for men to enjoy. I think that\'s one of the reasons why this movie is so perfect as a date movie. It has romance AND slapstick. I don\'t usually like slapstick that much, but it wasn\'t overdone and I can\'t think of any those scenes that didn\'t deliver laughs.<br /><br />I definitely recommend this movie. Th

In [62]:
for item in test_neg_reviews.take(2):
    print(item)
    print(' ')

(<tf.Tensor: shape=(), dtype=string, numpy=b'Naked Deanna Troi! <br /><br />Richie\'s brother Chuck (from "Happy Days") with a reverse mohawk!<br /><br />Death Wish 3 has all this and more, including one clever scene where Chuck Bronson\'s character sets up a mousetrap like device that brains a punk when he opens the window.<br /><br />Chuck also places a board with a bunch of nails on the floor near another window and smiles when he returns and sees bloody footprints leading away.<br /><br />All I can say about Death Wish 3 is that it is one of most incomprehensible "serious" movies that I have ever seen--loaded to the hilt with mayhem, and nearly zero police response, despite the use of military weapons to mow punks down by the score.<br /><br />As I understand it, Bronson disowned this film, but happily cashed the check from Golan Globus.<br /><br />If you enjoy quality movies, avoid this one, but if you\'re in for a cheap "Jackass" kind of thrill, check it out.'>, <tf.Tensor: shape

In [63]:
buffer_size = 10000
valid_dataset = valid_pos_reviews.concatenate(valid_neg_reviews).shuffle(buffer_size = buffer_size)

In [64]:
valid_batch = valid_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)

In [65]:
test_dataset = test_pos_reviews.concatenate(test_neg_reviews).shuffle(buffer_size)

In [66]:
test_batch = test_dataset.map(lambda X,y: 
                                (string_transform(X), y) ).batch(batch_size).prefetch(prefetch)